In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from resume import ResumeGenerator
from fake_contacts import FakeIdentity, fake
import time
import os
from datetime import datetime, date
from functools import wraps
from selenium.webdriver.common.action_chains import ActionChains
from sys import exit

In [2]:
fake_id = FakeIdentity()
resume = ResumeGenerator(fake_id.curriculum(), fake_id.contacts())
resume.generate()

'resume.pdf'

In [3]:
TIMEOUT = 30

def delay(t: int=1):
    def decorator_delay(f):
        @wraps(f)
        def wrapper(*args, **kwargs):
            time.sleep(t)
            return f(*args, **kwargs)
        return wrapper
    return decorator_delay


@delay()
def send_keys(el, keys):
    el.clear()
    el.send_keys(keys)

@delay(t=2)
def wait_element(t, locator):
    try:
        element = WebDriverWait(driver, t).until(
            EC.presence_of_element_located(locator)
        )
        element = WebDriverWait(driver, t).until(
            EC.element_to_be_clickable(locator)
        )
        element = WebDriverWait(driver, t).until(
            EC.visibility_of_element_located(locator)
        )
    except:
        print("timeout")
        driver.quit()
        exit(1)

In [4]:
driver = webdriver.Firefox()
driver.implicitly_wait(10)
driver.get("https://jobs.kellogg.com/job/Lancaster-Permanent-Production-Associate-Lancaster-PA-17601/817684800/")

# Cookie
driver.find_element(By.ID, "cookie-acknowledge").click()

# Apply now
driver.find_element(By.CSS_SELECTOR, 'button.btn.btn-primary.btn-large.btn-lg.dropdown-toggle').click()
driver.find_element(By.ID, 'applyOption-top-manual').click()

# Create new profile
wait_element(TIMEOUT, (By.CSS_SELECTOR, '.bottomLink a'))
try:
    element = WebDriverWait(driver, TIMEOUT).until(
        EC.invisibility_of_element_located((By.CLASS_NAME, "loading_indicator_layout_static"))
    )
except:
    print("timeout")
    driver.quit()
    exit(1)
driver.find_element(By.CSS_SELECTOR, '.bottomLink a').click()

# Fill new profile info
wait_element(TIMEOUT, (By.ID, "fbclc_userName"))
send_keys(driver.find_element(By.ID, "fbclc_userName"), fake_id.email)
send_keys(driver.find_element(By.ID, "fbclc_emailConf"), fake_id.email)
send_keys(driver.find_element(By.ID, "fbclc_pwd"), fake_id.pwd)
send_keys(driver.find_element(By.ID, "fbclc_pwdConf"), fake_id.pwd)
send_keys(driver.find_element(By.ID, "fbclc_fName"), fake_id.first_name)
send_keys(driver.find_element(By.ID, "fbclc_lName"), fake_id.last_name)
send_keys(driver.find_element(By.ID, "fbclc_phoneNumber"), fake_id.phone)

# Country dropdown
Select(driver.find_element(By.ID, "fbclc_ituCode")).select_by_value('US')
Select(driver.find_element(By.ID, "fbclc_country")).select_by_value('US')

# CAPTCHA
#driver.get("file:///home/luigi/Downloads/Career%20Opportunities%20Create%20an%20Account.html")
wait_element(TIMEOUT, (By.CSS_SELECTOR, 'iframe[title="reCAPTCHA"]'))
driver.execute_script("document.getElementById('dataPrivacyId').scrollIntoView(true);")
driver.switch_to.frame(driver.find_element(By.CSS_SELECTOR, 'iframe[title="reCAPTCHA"]'))
spinner = driver.find_element(By.CLASS_NAME, "recaptcha-checkbox-border")
spinner.click()
time.sleep(2)
driver.switch_to.default_content()
try:
    element = WebDriverWait(driver, 2*TIMEOUT).until_not(
        EC.visibility_of_element_located((By.CSS_SELECTOR, 'iframe[title="recaptcha challenge expires in two minutes"]'))
    )
except:
    print("timeout")
    driver.quit()
    exit(1)
time.sleep(2)
wait_element(TIMEOUT, (By.ID, "dataPrivacyId"))
driver.find_element(By.ID, "dataPrivacyId").click()
driver.find_element(By.ID, "dlgButton_20:").click()
driver.find_element(By.ID, "fbclc_createAccountButton").click()

# Load Resume
wait_element(TIMEOUT, (By.CLASS_NAME, "attachActions"))
driver.find_element(By.CLASS_NAME, "attachActions").click()
send_keys(driver.find_element(By.ID, "49:_file"), os.path.abspath('./resume.pdf')) # Upload Resume

In [5]:
# Profile Info
wait_element(TIMEOUT, (By.NAME, "currentTitle"))
send_keys(driver.find_element(By.NAME, "currentTitle"), fake_id.title) # Title

In [6]:
# Candidate specific information
wait_element(TIMEOUT, (By.NAME, "citizen"))
Select(driver.find_element(By.NAME, "citizen")).select_by_visible_text("Yes") # Citizen of US
driver.find_element(By.NAME, "expectedSalaryRange").send_keys("100000") # Salary expectation
Select(driver.find_element(By.NAME, "candidateSource")).select_by_value("3217000") # Source for job posting

In [7]:
# Profile Information
Select(driver.find_element(By.NAME, "presentEmployer")).select_by_value("299") # present employee

In [8]:
# Equal Employment Information
wait_element(TIMEOUT, (By.NAME, "custCountry"))
Select(driver.find_element(By.NAME, "custCountry")).select_by_value("3206850") # Country of origin
Select(driver.find_element(By.NAME, "veteranStatus")).select_by_value("299") # Veteran
Select(driver.find_element(By.NAME, "disabilityStatus")).select_by_value("299") # Disability
Select(driver.find_element(By.NAME, "custAge")).select_by_value("298") # Age 18+
Select(driver.find_element(By.NAME, "cust_sponsor")).select_by_value("299") # Sponsorship
Select(driver.find_element(By.NAME, "custPrev")).select_by_value("299") # Worked for kellogs
Select(driver.find_element(By.NAME, "custcontr")).select_by_value("299") # Contractor for Kellogs
Select(driver.find_element(By.NAME, "custRel")).select_by_value("299") # Relative for kellogs
Select(driver.find_element(By.NAME, "custAccom")).select_by_value("298") # Accomodation
Select(driver.find_element(By.NAME, "custgender")).select_by_value("71046" if fake_id.gender == 'F' else "71047") # Gender
Select(driver.find_element(By.NAME, "disabilityselection")).select_by_value("71280") # Disability 2

In [9]:
# Questionario
for i, rg in enumerate(driver.find_elements(By.CLASS_NAME, 'radioGroup')):
    for c in rg.find_elements_by_class_name('sfRadioInputField'):
        if i == 0 and c.text == 'Yes':
            c.find_element_by_tag_name('a').click()
        if i == 1 and c.text == '350 LBS':
            c.find_element_by_tag_name('a').click()
        if i == 2 and c.text == '800 LBS':
            c.find_element_by_tag_name('a').click()
        if i == 3 and c.text == 'Yes':
            c.find_element_by_tag_name('a').click()

/home/luigi/anaconda3/envs/kellogs/lib/python3.9/site-packages/selenium/webdriver/remote/webelement.py:464: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
/home/luigi/anaconda3/envs/kellogs/lib/python3.9/site-packages/selenium/webdriver/remote/webelement.py:341: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


In [10]:
[Select(el).select_by_value('296') for el in driver.find_elements(By.NAME, 'VFLD4')]
for div in driver.find_elements(By.CSS_SELECTOR, 'div.rcmFormSection.row'):
    try:
        h2 = div.find_element_by_tag_name('h2')
        if h2.text == 'Formal Education':
            print(h2.text)
            for i, uni in enumerate(div.find_elements_by_css_selector('.sectionContentOpen.sectionContent.rcmFormSectionContent div.row div.row')):
                type = Select(uni.find_element_by_name('VFLD3'))
                type.select_by_value("197" if 'B' in fake_id.education[i].type else "357")
                major = Select(uni.find_element_by_name('VFLD2'))
                index = [o.text for o in major.options].index(fake_id.education[i].major)
                if index:
                    major.select_by_index(index)
        elif h2.text == 'Candidate-Specific Information':
            print(h2.text)
            datepicker = div.find_element_by_css_selector('div.row span.datepicker input')
            datepicker.click()
            datepicker.clear()
            send_keys(datepicker, datetime.strftime(fake.date_between_dates(date(2022,1,1), date(2022,1,31)), '%m/%d/%Y')) # Date available
    except NoSuchElementException:
        continue

Formal Education


/home/luigi/anaconda3/envs/kellogs/lib/python3.9/site-packages/selenium/webdriver/remote/webelement.py:502: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")
/home/luigi/anaconda3/envs/kellogs/lib/python3.9/site-packages/selenium/webdriver/remote/webelement.py:227: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


Candidate-Specific Information


/home/luigi/anaconda3/envs/kellogs/lib/python3.9/site-packages/selenium/webdriver/remote/webelement.py:484: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


In [11]:
# Apply
for button in driver.find_elements(By.CLASS_NAME, 'rcmSaveButton'):
    if 'submit' in button.get_attribute('onclick'):
        button.click()

In [ ]:
driver.close()